In [1]:
import json, requests, jdc
import pandas as pd
import numpy as np

Create account from [API](https://www.ura.gov.sg/maps/api/#introduction) link

In [2]:
# this class will be defined across multiple cells
class call_api():
    def __init__(self, access_key=None):
        # variables
        self.access_key = None
        self.token = None
        self.header = None
        self.residential_property_transactions = None
        self.transactions = None
        self.street = None
        self.transaction_df = None
        self.median_rentals = None

        # access key from account creation
        self.access_key = '77144a24-fc1b-4358-b5b7-45d3a94a7aec' if not access_key else access_key

        # generate token from access key valid for one day
        url = 'https://www.ura.gov.sg/uraDataService/insertNewToken.action'
        header = {'AccessKey': f'{access_key}'}
        acc_info = requests.get(url, headers = header).json()
        assert acc_info['Status'] == 'Success', 'Too many tokens have been generated recently. Use another access_key'
        self.token = acc_info['Result']
        self.header = {'AccessKey': f'{self.access_key}', 'Token': self.token}

In [3]:
# instantiate for later use
api = call_api('89d0c48b-dcb2-4798-89f5-c2f4f5caddd8')

## Residential property transactions


This data service will return past 5 years of private residential property transaction records in JSON format. As transaction records > 5 years ago could be modified/aborted, we would advise to refresh your database on a daily basis and just retain the latest 5 years record for better accuracy.

Update Frequency: End of day of every Tuesday and Friday

- `project`	The name of the project
- `street`	The street name that the project is on.
- `marketSegment`	The market segment that the property falls in.
    - `CCR` – Core Central Region
    - `RCR` – Rest of Central Region
    - `OCR` – Outside Central Region
- `x`	The x coordinates of the address of the property in SVY21 format. Important: This is the location of the property and does not represent the location of the transacted unit.
- `y`	The y coordinates of the address of the property in SVY21 format. Important: This is the location of the property and does not represent the location of the transacted unit.
- `transaction`	An array of transactions for this property
    - `propertyType`	The property type of the transacted property. Note that there are properties with a mixture of property types.
        - `Strata` Detached
        - `Strata` Semidetached
        - `Strata` Terrace
        - `Detached`
        - `Semi`-detached
        - `Terrace`
        - `Apartment`
        - `Condominium`
        - `Executive` Condominium
    - `district`	The postal district that the transacted property falls in. Note that there are properties that fall across multiple postal district.
    - `tenure`	The tenure of the transacted property. Note that there are properties that have units with multiple tenures.
        - `Freehold`
        - `xx` yrs lease commencing from yyyy
    - `typeOfSale`	The type of sale
        - `1` – New Sale
        - `2` – Sub Sale
        - `3` – Resale
    - `noOfUnits`	The number of units in this transaction. The value for New Sale will always be 1. The value for Resale or Sub Sale could be greater than 1 depending on the number of units lodged for the caveat.
    - `price`	The transacted price nettPrice
    - `nettPrice`	The nett transacted price, excluding discounts if any. This field is only applicable for New Sale where discounts were given.
    - `area`	The land/floor area of the transacted unit in square metre.
    - `typeOfArea`	The type of area of the transacted unit.
        - `Strata`
        - `Land`
        - `Unknown`
    - `floorRange`	The floor range that the transacted unit falls within.
        - `-`
        - `B1`-B5
        - `B6`-B10
        - `01`-05
        - `06`-10
        ...
    - `contractDate`	The data of sale for New Sale records or option exercised date for Resale and Sub Sale records. Field is in format of mmyy e.g. 1215 represents Dec 2015.

In [4]:
%%add_to call_api
# above allows class to be defined across multiple cells
def call_residential_property_transactions(self, call=False):
    # call api once and keep in memory
    if call:
        url = 'https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=1'
        data = requests.get(url, headers = self.header).json()
    else:
        assert self.residential_property_transactions != None, 'call_residential_property_transactions() has already been called once'
        data = self.residential_property_transactions
    self.residential_property_transactions = data['Result']

In [5]:
# call once
api.call_residential_property_transactions(call=True)

In [6]:
# peek at the data
api.residential_property_transactions[:2]

[{'street': 'ZEHNDER ROAD',
  'project': 'LANDED HOUSING DEVELOPMENT',
  'transaction': [{'area': '524.3',
    'floorRange': '-',
    'noOfUnits': '1',
    'contractDate': '0220',
    'typeOfSale': '3',
    'price': '5500000',
    'propertyType': 'Semi-detached',
    'district': '05',
    'typeOfArea': 'Land',
    'tenure': 'Freehold'},
   {'area': '308',
    'floorRange': '-',
    'noOfUnits': '1',
    'contractDate': '0918',
    'typeOfSale': '3',
    'price': '5000000',
    'propertyType': 'Semi-detached',
    'district': '05',
    'typeOfArea': 'Land',
    'tenure': 'Freehold'},
   {'area': '314',
    'floorRange': '-',
    'noOfUnits': '1',
    'contractDate': '0618',
    'typeOfSale': '3',
    'price': '4750000',
    'propertyType': 'Semi-detached',
    'district': '05',
    'typeOfArea': 'Land',
    'tenure': 'Freehold'}],
  'marketSegment': 'RCR'},
 {'street': 'NEO PEE TECK LANE',
  'project': 'LANDED HOUSING DEVELOPMENT',
  'transaction': [{'area': '159.3',
    'floorRange': '

In [7]:
%%add_to call_api
def to_csv(self, save_path=None):
    '''
        converts json into a dataframe of useful information and saves it as a csv
        note stuff like 'project' is not considered useful and thus is not saved
    '''
    assert self.residential_property_transactions != None, 'call residential_property_transactions() first'
    assert save_path != None, 'please specify save_path'
    self.transactions = None

    for region in ['CCR', 'RCR', 'OCR']:
        for street in self.residential_property_transactions:
            if street['marketSegment'] == region:
                houses = street['transaction']
                for house in houses:
                    house['street'] = street['street']
                    house['project'] = street['project']
                    house['region'] = region
                    try:
                        house['x'] = street['x']
                        house['y'] = street['y']
                    except:
                        house['x'] = -1
                        house['y'] = -1

                if self.transactions != None:
                    self.transactions += houses
                else:
                    self.transactions = houses
                    
    # peek at first 2 house transactions
    print(self.transactions[:2])
    self.transactions = pd.DataFrame(self.transactions).set_index(['region', 'street'])
    self.transactions.to_csv(save_path)

In [8]:
api.to_csv('transactions.csv')
api.transactions.head()

[{'area': '226', 'floorRange': '01-05', 'noOfUnits': '1', 'contractDate': '0920', 'typeOfSale': '3', 'price': '3400000', 'propertyType': 'Condominium', 'district': '04', 'typeOfArea': 'Strata', 'tenure': '99 yrs lease commencing from 2007', 'street': 'COVE DRIVE', 'project': 'TURQUOISE', 'region': 'CCR', 'x': '28382.47067', 'y': '25008.33592'}, {'area': '224', 'floorRange': '01-05', 'noOfUnits': '1', 'contractDate': '0318', 'typeOfSale': '3', 'price': '3600000', 'propertyType': 'Condominium', 'district': '04', 'typeOfArea': 'Strata', 'tenure': '99 yrs lease commencing from 2007', 'street': 'COVE DRIVE', 'project': 'TURQUOISE', 'region': 'CCR', 'x': '28382.47067', 'y': '25008.33592'}]


area floorRange noOfUnits contractDate typeOfSale    price  \
region street                                                                  
CCR    COVE DRIVE  226      01-05         1         0920          3  3400000   
       COVE DRIVE  224      01-05         1         0318          3  3600000   
       COVE DRIVE  223      01-05         1         0118          3  3580000   
       COVE DRIVE  226      01-05         1         0516          3  3700000   
       COVE DRIVE  226      01-05         1         0219          3  3450000   

                  propertyType district typeOfArea  \
region street                                        
CCR    COVE DRIVE  Condominium       04     Strata   
       COVE DRIVE  Condominium       04     Strata   
       COVE DRIVE  Condominium       04     Strata   
       COVE DRIVE  Condominium       04     Strata   
       COVE DRIVE  Condominium       04     Strata   

                                              tenure    project            x  \
region street                                                                  
CCR    COVE DRIVE  99 yrs lease commencing from 2007  TURQUOISE  28382.47067   
       COVE DRIVE  99 yrs lease commencing from 2007  TURQUOISE  28382.47067   
       COVE DRIVE  99 yrs lease commencing from 2007  TURQUOISE  28382.47067   
       COVE DRIVE  99 yrs lease commencing from 2007  TURQUOISE  28382.47067   
       COVE DRIVE  99 yrs lease commencing from 2007  TURQUOISE  28382.47067   

                             y nettPrice  
region street                             
CCR    COVE DRIVE  25008.33592       NaN  
       COVE DRIVE  25008.33592       NaN  
       COVE DRIVE  25008.33592       NaN  
       COVE DRIVE  25008.33592       NaN  
       COVE DRIVE  25008.33592       NaN

## Private Non-Landed Residential Properties Median Rentals by Name

- `project`	The name of the project
- `street`	The street name that the project is on.
- `x`	The x coordinates of the address of the property in SVY21 format. Important: This is the location of the property and does not represent the location of the rented unit.
- `y`	The y coordinates of the address of the property in SVY21 format. Important: This is the location of the property and does not represent the location of the rented unit.
- `rentalMedian`	An array of median rentals for this `property`
    - `district`	The postal district that the transacted property falls in.
        - `01`
        - `02`
        - `03`
        - `04`
        - `...`
        - `28`
    - `refPeriod`	The reference period for the rental information. Field is in format of YYYYQQ e.g. 2011Q3 represents 2011 3rd quarter`.
    - `psf25`	The 25th percentile per square feet per month for the property for the reference period`.
    - `median`	The median per square feet per month for the property for the reference period.
    - `psf75`	The 75th percentile per square feet per month for the property for the reference period.

In [9]:
%%add_to call_api
def call_median_rentals(self):
    url = 'https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Rental_Median'
    data = requests.get(url, headers = self.header).json()
    self.median_rentals_data = data['Result']

In [10]:
api.call_median_rentals()
api.median_rentals_data[:2]

[{'rentalMedian': [{'refPeriod': '2018Q3',
    'psf75': 3.17,
    'median': 2.5,
    'psf25': 2.13,
    'district': '15'},
   {'refPeriod': '2020Q1',
    'psf75': 3,
    'median': 2.73,
    'psf25': 2.52,
    'district': '15'},
   {'refPeriod': '2019Q2',
    'psf75': 2.83,
    'median': 2.56,
    'psf25': 2.17,
    'district': '15'}],
  'street': 'ELLIOT ROAD',
  'project': 'ELLIOT AT THE EAST COAST'},
 {'rentalMedian': [{'refPeriod': '2018Q3',
    'psf75': 2.1,
    'median': 2.01,
    'psf25': 1.93,
    'district': '23'},
   {'refPeriod': '2020Q3',
    'psf75': 2.01,
    'median': 1.93,
    'psf25': 1.84,
    'district': '23'},
   {'refPeriod': '2019Q1',
    'psf75': 2.05,
    'median': 1.93,
    'psf25': 1.86,
    'district': '23'},
   {'refPeriod': '2019Q3',
    'psf75': 2.1,
    'median': 1.93,
    'psf25': 1.78,
    'district': '23'},
   {'refPeriod': '2020Q1',
    'psf75': 1.94,
    'median': 1.91,
    'psf25': 1.82,
    'district': '23'},
   {'refPeriod': '2019Q2',
    'psf75': 

In [11]:
%%add_to call_api
def median_to_csv(self, save_path):
    self.median_rentals = None
    for street in self.median_rentals_data:
        houses = street['rentalMedian']
        for house in houses:
            house['street'] = street['street']
            house['project'] = street['project']
            try:
                house['x'] = street['x']
                house['y'] = street['y']
            except:
                house['x'] = -1
                house['y'] = -1

        if self.median_rentals != None:
            self.median_rentals += houses
        else:
            self.median_rentals = houses
                    
    # peek at first 2 house median_rentals
    print(self.median_rentals[:2])
    self.median_rentals = pd.DataFrame(self.median_rentals).set_index(['street'])
    self.median_rentals.to_csv(save_path)

In [12]:
api.median_to_csv('median_rentals.csv')

[{'refPeriod': '2018Q3', 'psf75': 3.17, 'median': 2.5, 'psf25': 2.13, 'district': '15', 'street': 'ELLIOT ROAD', 'project': 'ELLIOT AT THE EAST COAST', 'x': -1, 'y': -1}, {'refPeriod': '2020Q1', 'psf75': 3, 'median': 2.73, 'psf25': 2.52, 'district': '15', 'street': 'ELLIOT ROAD', 'project': 'ELLIOT AT THE EAST COAST', 'x': -1, 'y': -1}]


In [13]:
api.median_rentals.head()

,refPeriod,psf75,median,psf25,district,project,x,y
street,,,,,,,,
ELLIOT ROAD,2018Q3,3.17,2.50,2.13,15,ELLIOT AT THE EAST COAST,-1,-1
ELLIOT ROAD,2020Q1,3.00,2.73,2.52,15,ELLIOT AT THE EAST COAST,-1,-1
ELLIOT ROAD,2019Q2,2.83,2.56,2.17,15,ELLIOT AT THE EAST COAST,-1,-1
HILLVIEW AVENUE,2018Q3,2.10,2.01,1.93,23,MERAWOODS,-1,-1
HILLVIEW AVENUE,2020Q3,2.01,1.93,1.84,23,MERAWOODS,-1,-1


## Private Residential Properties Rental Contract

- `project`	The name of the project
- `street`	The street name that the project is on.
- `x`	The x coordinates of the address of the property in SVY21 format. Important: This is the location of the property and does not represent the location of the transacted unit.
- `y`	The y coordinates of the address of the property in SVY21 format. Important: This is the location of the property and does not represent the location of the transacted unit.
- `rental`	An array of rental contracts for this property
    - `propertyType`	The property type of the transacted property. Note that there are properties with a mixture of property types.
        - `Non-landed Properties`
        - `Detached House`
        - `Semi-Detached House`
        - `Terrace House`
        - `Executive Condominium`
    - `district`	The postal district that the transacted property falls in. Note that there are properties that fall across multiple postal district.
        - `01`
        - `02`
        - `03`
        - `04`
        - `...`
        - `28`
    - `noOfBedRoom`	The number of bed rooms. Information is only available for non-landed property. Empty value for non-landed properties means that the information was not provided for this property.
    - `rent`	The monthly rent.
    - `areaSqft`	The floor area range of the rented property in square feet.
    - `areaSqm`	The floor area range of the rented property in square metre.
    - `leaseDate`	The lease commencement date of the rental. Field is in format of mmyy e.g. 0314 represents March 2014.

In [14]:
%%add_to call_api
def call_rental_name(self):
    url = 'https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Rental&refPeriod=14q1'
    data = requests.get(url, headers = self.header).json()
    self.rental_name_data = data['Result']

In [15]:
api.call_rental_name()
api.rental_name_data[:2]

[]